In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import seaborn as sns
import torch
from IPython.display import display

current_dir = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(current_dir, "../../../../"))

sys.path.append(os.path.join(PROJECT_ROOT, "shared_utilities"))
from metrics import *



In [2]:
ptid_list = [540, 544, 552, 559, 563, 567, 570, 575, 584, 588, 591, 596]

## **Dropout = 0.3**


In [3]:
dir_03 = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/dropout/03")

dict_03 = {}

detailed_results_df_03 = pd.DataFrame()

for ptid in ptid_list:
    df = pd.read_csv(os.path.join(dir_03, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_03[ptid] = df

    detailed_df = pd.read_csv(os.path.join(dir_03, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_03 = pd.concat([detailed_results_df_03, detailed_df], axis=0)
    detailed_results_df_03.reset_index(drop=True, inplace=True)
detailed_results_df_03.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,-0.226418,-0.083371,0.000000,0.000000,137.0,147.05199,0.0,0.000000,A,A,AP,eu,540
1,0,1,-0.112572,0.021719,0.022769,0.021018,145.0,154.43670,1.6,1.476944,A,A,AP,eu,540
2,0,2,-0.027188,0.076729,0.017077,0.011002,151.0,158.30228,1.2,0.773113,A,A,AP,eu,540
3,0,3,0.029735,0.140938,0.011385,0.012842,155.0,162.81430,0.8,0.902406,A,A,AP,eu,540
4,0,4,0.072427,0.160736,0.008538,0.003960,158.0,164.20552,0.6,0.278244,A,A,AP,eu,540


In [4]:
all_03_dfs = list(dict_03.values())

In [5]:
all_03_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_03_dfs]
aggregate_03_df = sum(all_03_value_only_dfs)
# add glycaemic region column
aggregate_03_df["Glycaemic Region"] = all_03_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_03_df = aggregate_03_df[["Glycaemic Region"] + list(aggregate_03_df.columns[:-1])]

aggregate_03_df["AP:EP"] = aggregate_03_df["AP"] / aggregate_03_df["EP"]

aggregate_03_df["AP%"] = aggregate_03_df["AP"] / aggregate_03_df["Count"] * 100
aggregate_03_df["BE%"] = aggregate_03_df["BE"] / aggregate_03_df["Count"] * 100
aggregate_03_df["EP%"] = aggregate_03_df["EP"] / aggregate_03_df["Count"] * 100

display(aggregate_03_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11861,375,885,13121,13.402260,90.397073,2.858014,6.744913
1,eu,306928,16562,4148,327638,73.994214,93.678999,5.054969,1.266031
2,hyper,172447,9397,6597,188441,26.140215,91.512463,4.986707,3.500830
3,overall,491236,26334,11630,529200,42.238693,92.826153,4.976190,2.197657


In [6]:
for ptid, df in dict_03.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_03[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1983,28,202,2213,9.816832,89.606869,1.265251,9.127881
1,eu,29394,2228,154,31776,190.870130,92.503776,7.011581,0.484642
2,hyper,12756,976,1071,14803,11.910364,86.171722,6.593258,7.235020
3,overall,44133,3232,1427,48792,30.927120,90.451303,6.624037,2.924660


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,136,2,22,160,6.181818,85.000000,1.250000,13.750000
1,eu,31050,1125,54,32229,575.000000,96.341804,3.490645,0.167551
2,hyper,15120,600,419,16139,36.085919,93.686102,3.717702,2.596196
3,overall,46306,1727,495,48528,93.547475,95.421200,3.558770,1.020030


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,81,2,13,96,6.230769,84.375000,2.083333,13.541667
1,eu,30087,1787,220,32094,136.759091,93.746495,5.568019,0.685486
2,hyper,25490,1587,829,27906,30.747889,91.342364,5.686949,2.970687
3,overall,55658,3376,1062,60096,52.408663,92.615149,5.617678,1.767173


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1867,127,127,2121,14.700787,88.024517,5.987742,5.987742
1,eu,34968,2616,948,38532,36.886076,90.750545,6.789162,2.460293
2,hyper,14182,957,584,15723,24.284247,90.199071,6.086625,3.714304
3,overall,51017,3700,1659,56376,30.751658,90.494182,6.563076,2.942742


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1222,24,74,1320,16.513514,92.575758,1.818182,5.606061
1,eu,33452,1586,551,35589,60.711434,93.995336,4.456433,1.548231
2,hyper,12728,699,352,13779,36.159091,92.372451,5.072937,2.554612
3,overall,47402,2309,977,50688,48.517912,93.517203,4.555319,1.927478


## **adjusted loss lowest**

In [7]:
low_adj_loss_dir = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/loss_function_weights_lowest")
dict_low_adj_loss = {}
detailed_results_df_low_adj_loss = pd.DataFrame()
for ptid in ptid_list:
    df = pd.read_csv(os.path.join(low_adj_loss_dir, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_low_adj_loss[ptid] = df

    detailed_df = pd.read_csv(os.path.join(low_adj_loss_dir, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_low_adj_loss = pd.concat([detailed_results_df_low_adj_loss, detailed_df], axis=0)
    detailed_results_df_low_adj_loss.reset_index(drop=True, inplace=True)
detailed_results_df_low_adj_loss.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,-1.236799,-1.491586,0.000000,0.000000,66.0,48.096024,0.0,0.000000,A,A,AP,hypo,540
1,0,1,-1.251030,-1.489212,-0.002846,0.000475,65.0,48.262817,-0.2,0.033358,A,A,AP,hypo,540
2,0,2,-1.236799,-1.497855,0.002846,-0.001728,66.0,47.655525,0.2,-0.121458,A,A,AP,hypo,540
3,0,3,-1.236799,-1.500269,0.000000,-0.000483,66.0,47.485855,0.0,-0.033933,A,A,AP,hypo,540
4,0,4,-1.251030,-1.488368,-0.002846,0.002380,65.0,48.322166,-0.2,0.167262,A,A,AP,hypo,540


In [8]:
all_low_adj_loss_dfs = list(dict_low_adj_loss.values())

In [9]:
all_low_adj_loss_value_only_df = [df.drop(columns=["Glycaemic Region"]) for df in all_low_adj_loss_dfs]
aggregated_low_adj_loss_df = sum(all_low_adj_loss_value_only_df)

aggregate_low_adj_loss_df = aggregated_low_adj_loss_df.copy()
aggregate_low_adj_loss_df["Glycaemic Region"] = all_low_adj_loss_dfs[0]["Glycaemic Region"]

aggregate_low_adj_loss_df = aggregate_low_adj_loss_df[["Glycaemic Region"] + list(aggregate_low_adj_loss_df.columns[:-1])]

aggregate_low_adj_loss_df["AP:EP"] = aggregate_low_adj_loss_df["AP"] / aggregate_low_adj_loss_df["EP"]

aggregate_low_adj_loss_df["AP%"] = aggregate_low_adj_loss_df["AP"] / aggregate_low_adj_loss_df["Count"] * 100
aggregate_low_adj_loss_df["BE%"] = aggregate_low_adj_loss_df["BE"] / aggregate_low_adj_loss_df["Count"] * 100
aggregate_low_adj_loss_df["EP%"] = aggregate_low_adj_loss_df["EP"] / aggregate_low_adj_loss_df["Count"] * 100

display(aggregate_low_adj_loss_df)



,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11871,414,836,13121,14.199761,90.473287,3.155247,6.371466
1,eu,306928,16481,4229,327638,72.576969,93.678999,5.030247,1.290754
2,hyper,172270,9538,6633,188441,25.971657,91.418534,5.061531,3.519935
3,overall,491069,26433,11698,529200,41.978885,92.794596,4.994898,2.210506


In [10]:
for ptid, df in dict_low_adj_loss.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_low_adj_loss[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1986,26,201,2213,9.880597,89.742431,1.174876,9.082693
1,eu,29470,2151,155,31776,190.129032,92.742951,6.769260,0.487790
2,hyper,12678,1010,1115,14803,11.370404,85.644802,6.822941,7.532257
3,overall,44134,3187,1471,48792,30.002719,90.453353,6.531808,3.014838


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,132,0,28,160,4.714286,82.500000,0.000000,17.500000
1,eu,31073,1102,54,32229,575.425926,96.413168,3.419281,0.167551
2,hyper,15098,619,422,16139,35.777251,93.549786,3.835430,2.614784
3,overall,46303,1721,504,48528,91.871032,95.415018,3.546406,1.038576


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,80,4,12,96,6.666667,83.333333,4.166667,12.500000
1,eu,30093,1764,237,32094,126.974684,93.765190,5.496354,0.738456
2,hyper,25472,1601,833,27906,30.578631,91.277861,5.737117,2.985021
3,overall,55645,3369,1082,60096,51.427911,92.593517,5.606030,1.800453


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1855,123,143,2121,12.972028,87.458746,5.799151,6.742103
1,eu,34968,2592,972,38532,35.975309,90.750545,6.726876,2.522579
2,hyper,14205,1015,503,15723,28.240557,90.345354,6.455511,3.199135
3,overall,51028,3730,1618,56376,31.537701,90.513694,6.616291,2.870016


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1172,49,99,1320,11.838384,88.787879,3.712121,7.500000
1,eu,33419,1632,538,35589,62.117100,93.902610,4.585687,1.511703
2,hyper,12808,637,334,13779,38.347305,92.953044,4.622977,2.423979
3,overall,47399,2318,971,50688,48.814624,93.511285,4.573074,1.915641


## **adjusted_Loss**

In [12]:
adj_loss_dir = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/loss_function_weights_lowest")
dict_adj_loss = {}
detailed_results_df_adj_loss = pd.DataFrame()
for ptid in ptid_list:
    df = pd.read_csv(os.path.join(adj_loss_dir, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_adj_loss[ptid] = df

    detailed_df = pd.read_csv(os.path.join(adj_loss_dir, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_adj_loss = pd.concat([detailed_results_df_adj_loss, detailed_df], axis=0)
    detailed_results_df_adj_loss.reset_index(drop=True, inplace=True)
detailed_results_df_adj_loss.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,-1.236799,-1.491586,0.000000,0.000000,66.0,48.096024,0.0,0.000000,A,A,AP,hypo,540
1,0,1,-1.251030,-1.489212,-0.002846,0.000475,65.0,48.262817,-0.2,0.033358,A,A,AP,hypo,540
2,0,2,-1.236799,-1.497855,0.002846,-0.001728,66.0,47.655525,0.2,-0.121458,A,A,AP,hypo,540
3,0,3,-1.236799,-1.500269,0.000000,-0.000483,66.0,47.485855,0.0,-0.033933,A,A,AP,hypo,540
4,0,4,-1.251030,-1.488368,-0.002846,0.002380,65.0,48.322166,-0.2,0.167262,A,A,AP,hypo,540


In [13]:
all_adj_loss_dfs = list(dict_adj_loss.values())

In [14]:
all_adj_loss_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_adj_loss_dfs]
aggregate_adj_loss_df = sum(all_adj_loss_value_only_dfs)
# add glycaemic region column
aggregate_adj_loss_df["Glycaemic Region"] = all_adj_loss_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_adj_loss_df = aggregate_adj_loss_df[["Glycaemic Region"] + list(aggregate_adj_loss_df.columns[:-1])]

aggregate_adj_loss_df["AP:EP"] = aggregate_adj_loss_df["AP"] / aggregate_adj_loss_df["EP"]

aggregate_adj_loss_df["AP%"] = aggregate_adj_loss_df["AP"] / aggregate_adj_loss_df["Count"] * 100
aggregate_adj_loss_df["BE%"] = aggregate_adj_loss_df["BE"] / aggregate_adj_loss_df["Count"] * 100
aggregate_adj_loss_df["EP%"] = aggregate_adj_loss_df["EP"] / aggregate_adj_loss_df["Count"] * 100
display(aggregate_adj_loss_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11871,414,836,13121,14.199761,90.473287,3.155247,6.371466
1,eu,306928,16481,4229,327638,72.576969,93.678999,5.030247,1.290754
2,hyper,172270,9538,6633,188441,25.971657,91.418534,5.061531,3.519935
3,overall,491069,26433,11698,529200,41.978885,92.794596,4.994898,2.210506


In [15]:
for ptid, df in dict_adj_loss.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_adj_loss[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1986,26,201,2213,9.880597,89.742431,1.174876,9.082693
1,eu,29470,2151,155,31776,190.129032,92.742951,6.769260,0.487790
2,hyper,12678,1010,1115,14803,11.370404,85.644802,6.822941,7.532257
3,overall,44134,3187,1471,48792,30.002719,90.453353,6.531808,3.014838


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,132,0,28,160,4.714286,82.500000,0.000000,17.500000
1,eu,31073,1102,54,32229,575.425926,96.413168,3.419281,0.167551
2,hyper,15098,619,422,16139,35.777251,93.549786,3.835430,2.614784
3,overall,46303,1721,504,48528,91.871032,95.415018,3.546406,1.038576


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,80,4,12,96,6.666667,83.333333,4.166667,12.500000
1,eu,30093,1764,237,32094,126.974684,93.765190,5.496354,0.738456
2,hyper,25472,1601,833,27906,30.578631,91.277861,5.737117,2.985021
3,overall,55645,3369,1082,60096,51.427911,92.593517,5.606030,1.800453


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1855,123,143,2121,12.972028,87.458746,5.799151,6.742103
1,eu,34968,2592,972,38532,35.975309,90.750545,6.726876,2.522579
2,hyper,14205,1015,503,15723,28.240557,90.345354,6.455511,3.199135
3,overall,51028,3730,1618,56376,31.537701,90.513694,6.616291,2.870016


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1172,49,99,1320,11.838384,88.787879,3.712121,7.500000
1,eu,33419,1632,538,35589,62.117100,93.902610,4.585687,1.511703
2,hyper,12808,637,334,13779,38.347305,92.953044,4.622977,2.423979
3,overall,47399,2318,971,50688,48.814624,93.511285,4.573074,1.915641


## **Comparison**

In [16]:
display(aggregate_03_df)

display(aggregate_low_adj_loss_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11861,375,885,13121,13.402260,90.397073,2.858014,6.744913
1,eu,306928,16562,4148,327638,73.994214,93.678999,5.054969,1.266031
2,hyper,172447,9397,6597,188441,26.140215,91.512463,4.986707,3.500830
3,overall,491236,26334,11630,529200,42.238693,92.826153,4.976190,2.197657


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11871,414,836,13121,14.199761,90.473287,3.155247,6.371466
1,eu,306928,16481,4229,327638,72.576969,93.678999,5.030247,1.290754
2,hyper,172270,9538,6633,188441,25.971657,91.418534,5.061531,3.519935
3,overall,491069,26433,11698,529200,41.978885,92.794596,4.994898,2.210506
